<a href="https://colab.research.google.com/github/Saku052/YoutubeindustryReserch/blob/main/Youtube_SDIndustry_Api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
from googleapiclient.discovery import build
from pyasn1.compat.integer import null
from google.colab import files
import pandas as pd

In [3]:
api_key = ''

#チャンネルリスト
AIChannels = ['UCtLc0LqPp8aq4_cvjBDVfCw', #ぺんたろう
              'UCtw4S54qOeBW1inR5--jSFQ', #ぷりずんだAI
              'UCAajXFu19cCUvSJW2FdHcwg', #しろぼうVtuber
              'UCYqTtuRm0H3roAL0iJd1CaA', #キシリのお絵かきチャンネル
              'UCmqz9u0Fp099PZr3Bb_GZ9A', #動き続けるCAN
              'UClz-2IhWHgFNw7gnHe6Xgyw'  #だだっこぱんだ
              ]

GameDev = ['UCY_mh7pxNlNUR8-YYf_s4uA', #ゲームの作り方ch
           'UCVzrBUq93iPUzHX2kqG-xuA', #Unity幼稚園
           'UCqD8vOZg1hmmgmC6zD-dKLg', #Unity小枝チャンネル
           'UCjPKjWOUTCOS_5olrDHpjqQ', #ITeens Lab
           'UCiIbLpncjzahHsp8cokG56g', #八ツ橋まろん
           'UCMB4OYmMrzbxnh6ZpDP2pmQ'  #YousukouTV
           ]

Photoshop = ['UC7fdDb7fgdgZC_XRHfQkbFw', #Photoshop Elements講座
             'UCtt4e-XeVcMB3LV07ya49RA', #初心者の為のphotoshop講座
             'UC9ojHkqpODljabT7u5nLJGg', #Photo Graphico Art 
             'UCwXj1CsYY7rUyytzdHGFs0w', #石原 誠之 / STACK ONLINE
             'UCz2pXojCQO5HVHrGA1jCw4g', #プロのデザイン講座うえのくん
             'UCEVRlepbWNTgAwY23Tqbm5A'  #emi-lab Art days
             ]

VideoEdit = ['UCZqS1Tbk1dQtFxfxGdPJjdw', #夏目 Premiere Pro
             'UCSIqvpAXIU8TLwWqn5Ryuug', #動画編集で月５万
             'UCP4d4UqNI_x42q6efNe3SDQ', #NEGOTO VIDEO
             'UCCtDvKGQVKGljYZtb6ixfeQ', #InnoRise
             'UCzZu7Vmi-gfSnOs3kBoUz7g', #HIROvideo
             'UCLnBmbgA9yctqhLX1Rxq_Ww'  #ToolsNotesChannel
             ]

youtube = build('youtube', 'v3', developerKey = api_key)

## チャンネルの取得

In [4]:
def get_channel_stats(youtube, channel_id):
  all_data = []
  
  request = youtube.channels().list(
      part="snippet,contentDetails,statistics",
        id= ','.join(channel_id)
  )
  response = request.execute()

  for i in range(len(response['items'])):
    data = dict(Channel_name = response['items'][i]['snippet']['title'],
                Subscribers = response['items'][i]['statistics']['subscriberCount'],
                Views = response['items'][i]['statistics']['viewCount'],
                Total_videos = response['items'][i]['statistics']['videoCount'],
                playlist_id = response['items'][i]['contentDetails']['relatedPlaylists']['uploads'])
    all_data.append(data)

  return all_data

In [10]:
##チャンネルデータ取得
ai_data = pd.DataFrame(get_channel_stats(youtube, AIChannels))
game_data = pd.DataFrame(get_channel_stats(youtube, GameDev))
photo_data = pd.DataFrame(get_channel_stats(youtube, Photoshop))
video_data = pd.DataFrame(get_channel_stats(youtube, VideoEdit))

##データフレームのカテゴライズ
ai_data.insert(0, 'Channel_Category', 'AIChannels')
game_data.insert(0, 'Channel_Category', 'GameDev')
photo_data.insert(0, 'Channel_Category', 'Photoshop')
video_data.insert(0, 'Channel_Category', 'VideoEdit')

##データフレームの結合
Channel_data = pd.concat([ai_data, game_data, photo_data, video_data], ignore_index= True)
Channel_data.head(10)

,Channel_Category,Channel_name,Subscribers,Views,Total_videos,playlist_id
0,AIChannels,だだっこぱんだ,2280,633267,91,UUlz-2IhWHgFNw7gnHe6Xgyw
1,AIChannels,しろぼう【Vtuber講座】,4190,408854,28,UUAajXFu19cCUvSJW2FdHcwg
2,AIChannels,キシリのお絵描きチャンネル,1580,146756,149,UUYqTtuRm0H3roAL0iJd1CaA
3,AIChannels,ぷりずんだAI,3560,53414,9,UUtw4S54qOeBW1inR5--jSFQ
4,AIChannels,ペンたろうのAIイラストお絵描きチャンネル,565,43636,21,UUtLc0LqPp8aq4_cvjBDVfCw
5,AIChannels,動き続けるCAN,397,48303,26,UUmqz9u0Fp099PZr3Bb_GZ9A
6,GameDev,八ツ橋まろん,1540,78506,85,UUiIbLpncjzahHsp8cokG56g
7,GameDev,ITeens Lab,2240,213026,332,UUjPKjWOUTCOS_5olrDHpjqQ
8,GameDev,【Unity初心者向け】ゲームの作り方ch,561,22444,31,UUY_mh7pxNlNUR8-YYf_s4uA
9,GameDev,ヨウスコウTV【ゲーム制作中】,3020,194970,85,UUMB4OYmMrzbxnh6ZpDP2pmQ


## 動画データの取得

In [18]:
## チャンネルの全ての動画を取得する
def get_video_ids(youtube, playlist_id):
  video_ids = []

  request = youtube.playlistItems().list(
      part = 'contentDetails',
      playlistId = playlist_id,
      maxResults = 30)
  response = request.execute()

  for i in range(len(response['items'])):
    video_ids.append(response['items'][i]['contentDetails']['videoId'])

  return video_ids

In [67]:
## 動画のメタデータを取得する
def get_video_details(youtube, video_ids):
  all_video_stats = []
  request = youtube.videos().list(
      part='snippet,statistics',
      id=','.join(video_ids)
  )
  response = request.execute()

  for video in response['items']:
    try:
      Likes = video['statistics']['likeCount']
    except:
      Likes = null

    try:
      Comments = video['statistics']['commentCount']
    except:
      Comments = null

    video_stats = dict(Title = video['snippet']['title'],
                       published_date = video['snippet']['publishedAt'],
                       Views = video['statistics']['viewCount'],
                       Likes = Likes,
                       Comments = Comments)
    all_video_stats.append(video_stats)

  return all_video_stats

In [68]:
## 
def get_video_list(ch_data):
  df = pd.DataFrame()
  for i in range(len(ch_data)):
    Vid_id_list = get_video_ids(youtube, ch_data.iloc[i]['playlist_id'])
    Temp_df = pd.DataFrame(get_video_details(youtube, Vid_id_list))
    Temp_df.insert(0, 'Channel_Category', ch_data.iloc[i]['Channel_Category'])
    Temp_df.insert(1, 'Channel_name', ch_data.iloc[i]['Channel_name'])
    df = df.append(Temp_df, ignore_index = True)
  return df

In [69]:
Video_csv = get_video_list(Channel_data)
Video_csv.tail(5)

,Channel_Category,Channel_name,Title,published_date,Views,Likes,Comments
663,VideoEdit,NEGOTO VIDEO,【After Effects】絶対使えるようになる！何処よりも分かり易いエクスプレッション...,2022-04-22T08:23:36Z,420,23,0
664,VideoEdit,NEGOTO VIDEO,【AfterEffects】ペンキを塗った様におしゃれに動画を切り替えるペンキブラシ風...,2022-04-18T08:54:00Z,1223,29,0
665,VideoEdit,NEGOTO VIDEO,【AfterEffects】Media Encoder（メディアエンコーダー）を使って動...,2022-04-11T07:30:02Z,819,24,0
666,VideoEdit,NEGOTO VIDEO,【After Effects】めっちゃ簡単でかっこいいテキストストロークアニメーションの作...,2022-04-01T08:30:06Z,641,29,2
667,VideoEdit,NEGOTO VIDEO,【After Effects】絶対見たことある！動画やスライドショーにめっちゃ使えるキラキ...,2022-03-17T05:20:53Z,762,23,0


In [70]:
Video_csv.to_csv('Youtube_SDIndustry.csv', encoding = 'utf-8-sig') 
files.download('Youtube_SDIndustry.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>